In [1]:
import google.generativeai as genai
import os
import json

/home/rrenaud/miniconda3/envs/med_bib/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GEMINI_API_KEY = open(os.path.expanduser('~/.gemini_api_key'), 'r').read().strip()
genai.configure(api_key=GEMINI_API_KEY)
safety_settings = [{'category': c, 'threshold': 'BLOCK_NONE'} for c in
                    ['HARM_CATEGORY_DANGEROUS', 'HARM_CATEGORY_HARASSMENT', 'HARM_CATEGORY_HATE_SPEECH',
                     'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'HARM_CATEGORY_DANGEROUS_CONTENT']]
flash = genai.GenerativeModel('gemini-1.5-flash', safety_settings=safety_settings)

In [3]:
stories = json.load(open('spanish_stories/stories_00.json', 'r'))

In [4]:
print(stories[0]['story'])

Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despacio. Él no quería asustarlos. Él era feliz. Él sabía que su sueño había sido real. 



In [12]:
prompt = "Translate the following story to English:\n\n" + stories[1]['story']
print(prompt)
# resp = flash.generate_content(prompt)
print(resp.text)


Translate the following story to English:

Una niña llamada Lily estaba enferma.  Tenía mucho frío.  Mamá dijo, "Lily, ponte tus botas".  Lily dijo, "¡No quiero!"  Mamá dijo, "Si te pones las botas, te sentirás mejor".  Lily dijo, "¡Ganaré!"  Lily se puso las botas.  ¡Se sintió mejor!

Lily fue al parque.  Vio a un perro.  El perro dijo, "Woof!"  Lily dijo, "¡Hola!"  El perro corrió.  Lily corrió también.  ¡Se divirtió mucho!

Después, Lily regresó a casa.  Mamá la abrazó.  Mamá dijo, "¡Me alegro de que te sientas mejor!"  Lily dijo, "¡Yo también!"  Lily se sentó en el sofá.  Mamá le dio un beso.  Lily se sintió muy feliz.

Lily se fue a la cama.  Mamá le leyó una historia.  Lily cerró los ojos.  Mamá dijo, "Buenas noches, mi amor."  Lily dijo, "Buenas noches, mamá."  Lily soñó con perros y botas.  Fue una buena noche.

A little girl named Lily was sick.  She was very cold.  Mom said, "Lily, put on your boots."  Lily said, "I don't want to!"  Mom said, "If you put on your boots, you'll